# Part 3: Huffman Coding

In these questions, we'll utilize a full English text to create frequency tables for the English language.

We will use a book text to calculate frequencies of all English alphabet letters, including spaces.

Read the book provided through Moodle using the `read_text` function.

In [24]:
def read_text(file):
    f = open(file, 'r', encoding='utf-8')
    text0 = f.read()
    f.close()

    return clean_raw_text(text0)

def clean_raw_text(text0):
  text0 = text0.lower()

  legalABC = 'abcdefghijklmnopqrstuvwxyz '
  text = ''
  last_was_space = True

  for ch in text0:
      if ch == '\n':
          ch = ' '
      if ch in legalABC:
          if ch != " ":
              text = text + ch
              last_was_space = False
          if ch == " " and last_was_space == False:
              text= text + ch
              last_was_space = True
  return text

text = read_text('book-hw1.txt')

## Question 6 (10 points):

Create the frequency table and print it.

Count only lowercase letters '**a**' through '**z**' and the **space** character.

**Store frequencies as percentages (like 0.8% or 0.008), not their count.**

Example: {' ': 17, 'a': 8.1, 'b': 1.4, ..., 'z': 0.07 }

In [25]:
def freq_table(text):
    """
    Calculate the frequency of each character in the text as a percentage of the total characters.
    Sort the results alphabetically.
    Args:
      text (str): your text file.

    Returns:
      freq_table (dict): A dictionary containing characters as keys and their corresponding frequencies as values.
                         The dictionary is sorted alphabetically by character.
    """
    characters_count_dict = dict()

    for c in text:
      characters_count_dict[c] = characters_count_dict.get(c, 0) + 1

    return {
        charachter: count / len(text) for charachter, count in characters_count_dict.items()
    }

In [26]:
# print your table here
freq_table(text)

{'b': 0.011766921616635027,
 'u': 0.022405485452184796,
 'r': 0.04396761929841813,
 'n': 0.05864127056086215,
 'i': 0.05447784702001877,
 'g': 0.019833763064866405,
 ' ': 0.187677399676993,
 'd': 0.041192559172115395,
 'a': 0.06797980622913496,
 'y': 0.01684203157735084,
 'l': 0.034855755208029505,
 'h': 0.054362844528298114,
 't': 0.07387993406523809,
 'j': 0.0009600208004506765,
 'c': 0.018282062778026856,
 'k': 0.008250178753873,
 'o': 0.05942795427234257,
 'p': 0.011928591786155367,
 'e': 0.09820046100998855,
 'w': 0.020137102970564362,
 's': 0.048201044355961044,
 'q': 0.0007533496559092113,
 'v': 0.006666811114240809,
 'f': 0.017437044469296836,
 'm': 0.02012543605111444,
 'z': 0.0006783480308740023,
 'x': 0.0010683564810570895}

Following is a table frequency of general English text that includes only the letters '**a**' through '**z**', make sure that your table looks something like this:

Letter | Frequency
--- | :---:
SPACE |
a | 8.167%
b | 1.492%
c | 2.782%
d | 4.253%
e | 12.702%
f | 2.228%
g | 2.015%
h | 6.094%
i | 6.966%
j | 0.253%
k | 1.772%
l | 4.025%
m | 2.406%
n | 6.749%
o | 7.507%
p | 1.929%
q | 0.095%
r | 5.987%
s | 6.327%
t | 9.056%
u | 2.758%
v | 0.978%
w | 2.36%
x | 0.25%
y | 1.974%
z | 0.074%

## Question 7 (10 points):

Write code to construct a Huffman Tree representing English alphabet letters, including spaces.

The goal of this exercise is to create an encoding table that looks something like:

a : 0000

b : 00010

c : 00011

...

**Feel free to make changes and adjustments to the code.**

In [27]:
import heapq

class NodeTree(object):
    """
    Represents a node in the Huffman Tree.

    Args:
    left (NodeTree): The left child node (default is None).
    right (NodeTree): The right child node (default is None).

    Methods:
    children(): Returns the left and right children of the node.
    str(): Returns a string representation of the node, including its left and right children.
    """
    def __init__(self, value=None, left=None, right=None):
        self._left = left
        self._right = right
        self._value = value

    def value(self):
        return self._value

    def children(self):
        return self._left, self._right

    def __str__(self):
        if not self._left and not self._right:
          return f"({self._value})"
        return f"({self._left} <-- () --> {self._right})"

    def __lt__(self, other):
        return True


def huffman_code_tree(node: NodeTree, binString=''):
    """
    Recursively generates Huffman codes for characters in the Huffman Tree.

    Args:
    node (NodeTree): The current node in the Huffman Tree.
    binString (str, optional): The binary string for recursion.

    Returns:
    codes (dict): A dictionary of characters and their corresponding Huffman codes.
    """
    left, right = node.children()

    if not left and not right:
      return {
          node.value(): binString
      }

    left_dict = huffman_code_tree(left,  binString + "0")
    right_dict = huffman_code_tree(right, binString + "1")

    return {
        **left_dict,
        **right_dict
    }


def make_tree(nodes):
    """
    Constructs the Huffman Tree from a list of nodes representing characters and their frequencies.

    Args:
    nodes (list): A decending sorted list of tuples with character-frequency pairs.

    Returns:
    root (NodeTree): The root node of the Huffman Tree.
    """
    heap = [(freq, NodeTree(char)) for char, freq in nodes]
    while len(heap) > 1:
        left_freq, left_node = heapq.heappop(heap)
        right_freq, right_node = heapq.heappop(heap)
        combined_freq = left_freq + right_freq
        combined_node = NodeTree(value=None, left=left_node, right=right_node)
        heapq.heappush(heap, (combined_freq, combined_node))
    return heap[0][1]


def init_huffman(text, print_code=False):
    """
    Initializes Huffman encoding for a given text.

    Args:
    text (str): The input text to be encoded.
    print_code (bool, optional): Whether to print Huffman codes.

    Returns:
    encoding (dict): A dictionary of characters and their Huffman codes.
    freq (list): A sorted list of tuples representing character frequencies.
    """
    freq = sorted(freq_table(text).items(), key=(lambda entry: entry[1]))
    encoding_tree = make_tree(freq)

    encoding = huffman_code_tree(encoding_tree)

    if print_code:
      print(encoding)

    return encoding, freq


init_huffman(text)

({' ': '00',
  's': '0100',
  'h': '0101',
  'i': '0110',
  'b': '011100',
  'p': '011101',
  'y': '011110',
  'f': '011111',
  'n': '1000',
  'o': '1001',
  'a': '1010',
  'l': '10110',
  'c': '101110',
  'k': '1011110',
  'z': '1011111000',
  'q': '1011111001',
  'j': '1011111010',
  'x': '1011111011',
  'v': '10111111',
  't': '1100',
  'g': '110100',
  'm': '110101',
  'd': '11011',
  'w': '111000',
  'u': '111001',
  'r': '11101',
  'e': '1111'},
 [('z', 0.0006783480308740023),
  ('q', 0.0007533496559092113),
  ('j', 0.0009600208004506765),
  ('x', 0.0010683564810570895),
  ('v', 0.006666811114240809),
  ('k', 0.008250178753873),
  ('b', 0.011766921616635027),
  ('p', 0.011928591786155367),
  ('y', 0.01684203157735084),
  ('f', 0.017437044469296836),
  ('c', 0.018282062778026856),
  ('g', 0.019833763064866405),
  ('m', 0.02012543605111444),
  ('w', 0.020137102970564362),
  ('u', 0.022405485452184796),
  ('l', 0.034855755208029505),
  ('d', 0.041192559172115395),
  ('r', 0.04396761

## Question 8 (5 points):
* Encode the first 1000 letters from our text using Huffman coding and print your encoding table.
Determine the number of bits required for Huffman encoding.
Compare this to storing letters in ASCII code, which needs 8 bits per character (totaling 8000 bits for 1000 characters).

* Encode this message in Huffman code: **"The rat and the bear had a date. The rat had a bad hat. The bear had a red beard. Then, the rat had tea and the bear had naan."** Print your encoding table. Determine the number of bits required for Huffman encoding.
Compare this to storing letters in ASCII code, which needs 8 bits per character.

* Answer the open question below.

* **Note:** Don't forget to remove punctuations and use only lowercased letters.

In [28]:
def calculate_encoded_text_bit_size(text: str):
  encoding_dict, frequencies = init_huffman(text)
  bit_size = 0

  for character, frequency in frequencies:
    bit_size += frequency * len(text) * len(encoding_dict[character])

  return bit_size

first_1000_encoded_bit_size = calculate_encoded_text_bit_size(text[:1000])
huffman_vs_ascii_ratio = first_1000_encoded_bit_size / 8000
print(f"Encoding the first 1000 characters will take in Huffman encoding: {first_1000_encoded_bit_size} bits, 8000 bits in ASCII. Ratio: {huffman_vs_ascii_ratio}")

message = "The rat and the bear had a date. The rat had a bad hat. The bear had a red beard. Then, the rat had tea and the bear had naan"
clean_message = clean_raw_text(message)
clean_message_encoded_bit_size = calculate_encoded_text_bit_size(clean_message)
clean_message_ascii_bit_size = len(clean_message) * 8

huffman_vs_ascii_ratio = clean_message_encoded_bit_size / clean_message_ascii_bit_size
print(f"Encoding the message characters will take in Huffman encoding: {clean_message_encoded_bit_size} bits, {clean_message_ascii_bit_size} bits in ASCII. Ratio: {huffman_vs_ascii_ratio}")



Encoding the first 1000 characters will take in Huffman encoding: 4146.0 bits, 8000 bits in ASCII. Ratio: 0.51825
Encoding the message characters will take in Huffman encoding: 362.0 bits, 968 bits in ASCII. Ratio: 0.3739669421487603


### Explain in your own words why using Huffman coding is more efficient in the second encoding of the short sentence compared to encoding the entire text.

The main cause for that difference is that the short sentence only contains 9 different characters while the long text contains all the english letters and space. Meaning the maximum bits used to encode a single character in the short sentence is much smaller than the maximum bits used to encode a single character in the long text. This will result in less bits in total being used.
Other parameter that can effect the differences is the distribution of characters being used, for example a string contains 1000 times the character 'a' and all the other english characters are only written one time will also be extremly efficient in storage.

## Question 9 (5 points):
Here is a list of words:

`Hi, For, Forest, False, Falls, Fall, Friend, Friends, High`

Create binary representations using the Huffman code method.

Build the Huffman Tree for encoding and provide the binary representation for the message "False Friends Fall High."

Use separators between words for clarity.

In [29]:
encoding_dict, _f = init_huffman("".join(["Hi", "For", "Forest", "False", "Falls", "Fall", "Friend", "Friends", "High"]))

print("Encoded 'False Friends Fall High':")
print("".join(encoding_dict[c] if not c == " " else c for c in "False Friends Fall High"))



Encoded 'False Friends Fall High':
1101010100011010 11000011110101110011101011 1101010100100 1011111110011110110
